<a href="https://colab.research.google.com/github/ShaifaliKhulbe/Masters-Thesis-Authorship-Attributon/blob/main/French_Baseline%2BPOS_(Universal_tagset_nltk).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import nltk
import collections
from nltk import word_tokenize
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')

**Load Data**

In [ ]:
import nltk
from itertools import chain
from nltk.util import ngrams
from collections import Counter

# load data
df = pd.read_csv('200_chunks_French.csv')
test_df = pd.read_csv('Test_200_chunks_French.csv')


**top N POS NGRAMs for each author**

In [ ]:
#define the dictionary to store all top ngrams for each author
top_ngrams = {}

#define the number of top pos ngrams to calculate
num = 500 

# calculate the pos ngrams for each author and store them in pos_ngrams_by_author
for author in df['Author_name'].unique():
    author_data = df[df['Author_name'] == author]
    sentences = list(author_data['chunks'])

    all_pos_1grams = []
    all_pos_2grams = []
    all_pos_3grams = []

    for sentence in sentences:

        # tokenize the sentence
        tokens = nltk.word_tokenize(sentence, language='french') #TOKENIZER
        # tag the tokens with POS tags
        pos_tags = nltk.pos_tag(tokens, tagset='universal') #POS TAGGING

        # create POS n-grams of size n
        pos_unigram_list = ngrams([tag for _, tag in pos_tags], 1)
        pos_bigram_list = ngrams([tag for _, tag in pos_tags], 2)
        pos_trigram_list = ngrams([tag for _, tag in pos_tags], 3)

        # add the POS n-grams to the list
        all_pos_1grams.extend(pos_unigram_list)
        all_pos_2grams.extend(pos_bigram_list)
        all_pos_3grams.extend(pos_trigram_list)
      
  
    # count the frequency of each POS n-gram
    pos_1gram_counts = nltk.FreqDist(all_pos_1grams)
    pos_2gram_counts = nltk.FreqDist(all_pos_2grams)
    pos_3gram_counts = nltk.FreqDist(all_pos_3grams)
    
    # return the top num POS n-grams

    top_unigrams= [ngram for ngram, count in pos_1gram_counts.most_common(num)]

    top_bigrams= [ngram for ngram, count in pos_2gram_counts.most_common(num)]

    top_trigrams= [ngram for ngram, count in pos_3gram_counts.most_common(num)]

    top_ngrams[author] = top_trigrams + top_bigrams + top_unigrams

#print(top_ngrams)

In [ ]:
# Find the name of the first key in the dictionary
first_key = list(top_ngrams.keys())[0]

# Initialize the common_values set using the first key
common_ngrams = set(top_ngrams[first_key])

for key in top_ngrams:
    common_ngrams = common_ngrams.intersection(top_ngrams[key])

print(len(common_ngrams))

# Create a list of all values from all keys minus the common values
all_ngrams = []

for key in top_ngrams:
    all_ngrams.extend(top_ngrams[key])
    
print(len(all_ngrams))

not_common_ngrams = set(all_ngrams).difference(common_ngrams)
not_common_ngrams_list = list(not_common_ngrams)


505
3148


In [ ]:
#Finding top 100 most frequent words
def find_top_function_words(data, num):
    
    # tokenize each sentence and create a list of all words
    words = []
    for sentence in data['chunks']:
        tokens = nltk.word_tokenize(sentence.lower(), language='french') #TOKENIZER
        words.extend(tokens)

    # count the frequency of each word in the entire CSV
    word_freq = Counter(words)
    
    # sort the dictionary by frequency in descending order
    sorted_word_freq = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)

    # extract the 100 most frequent stopwords
    
    top_stop_words = [word[0] for word in sorted_word_freq if word[0] in nltk.corpus.stopwords.words('french')][:100] #STOPWORDS

    return top_stop_words
       
#NEW    
top_function_words = find_top_function_words(df, 100)

In [ ]:
# define a function to create the feature vector for a given sentence
def create_feature_vector(sentence):
    # tokenize the sentence
    tokens = nltk.word_tokenize(sentence, language='french') #TOKENIZER
    
    # tag the tokens with POS tags
    pos_tags = nltk.pos_tag(tokens, tagset='universal') #POS TAGGING
    
    # create bigram, trigram, and four-gram POS sequences
    unigram_pos_seqs = ngrams([tag for _, tag in pos_tags], 1)
    bigram_pos_seqs = ngrams([tag for _, tag in pos_tags], 2)
    trigram_pos_seqs = ngrams([tag for _, tag in pos_tags], 3)

    # concatenate the three n-gram sequences into one
    all_ngram_pos = chain(unigram_pos_seqs, bigram_pos_seqs, trigram_pos_seqs)

    # create a list to store the counts of each n-gram in the not_common_ngrams_list
    pos_counts = []

    # concatenate the three n-gram sequences into one
    all_ngram_pos = chain(unigram_pos_seqs, bigram_pos_seqs, trigram_pos_seqs)


    # Converting all_ngram_pos to a list and printing the first 10 n-grams
    all_ngrams_list = list(all_ngram_pos)


    pos_counts = []
    
    for pos_ngram_list, top_pos_ngrams in zip([all_ngrams_list],
                                              [not_common_ngrams_list]):
        for pos_ngram in top_pos_ngrams:
            count = 0
            for top_pos in pos_ngram_list:
                if pos_ngram == top_pos:
                    count += 1
            pos_counts.append(count)

    
    # Tokenize the sentence into lowercase words
    tokens = nltk.word_tokenize(sentence.lower(), language='french') #TOKENIZER

    # Count the occurrences of each word in the sentence
    word_counts = Counter(tokens)

    # Create a feature vector based on the number of occurrences of the top 100 stopwords
    top_function_vector = []
    for stop_word in top_function_words:
        count = word_counts.get(stop_word, 0)
        top_function_vector.append(count)
           
    # count the occurrences of each punctuation mark
    punctuation_marks = ['.', ',', ';', ':', '?', '!', '«', '»', '-', '–', '—', '(', ')', '[', ']', '{', '}', '/', '\\', '...', "'", '"']
    
    punctuation_counts = []
    for mark in punctuation_marks:
        count = sentence.count(mark)
        punctuation_counts.append(count)

    #feature_vector = punctuation_counts + top_function_vector + pos_counts

    # create scaler objects for each feature type
    punc_scaler = StandardScaler()
    top_func_scaler = StandardScaler()
    pos_scaler = StandardScaler()

   # convert to NumPy arrays and reshape to have one column
    punctuation_counts_reshaped = np.array(punctuation_counts).reshape(-1, 1)
    top_function_vector_reshaped = np.array(top_function_vector).reshape(-1, 1)
    pos_counts_reshaped = np.array(pos_counts).reshape(-1, 1)
    
    # fit and transform each feature type separately
    punc_counts_scaled = punc_scaler.fit_transform(punctuation_counts_reshaped)
    top_func_scaled = top_func_scaler.fit_transform(top_function_vector_reshaped)
    pos_counts_scaled = pos_scaler.fit_transform(pos_counts_reshaped)
    
    # concatenate the scaled features
    feature_vector = np.concatenate((punc_counts_scaled, top_func_scaled, pos_counts_scaled), axis=0)
    
    # convert back to a 1D array and return
    return feature_vector.flatten()

# create feature vectors for each sentence
df['features'] = df['chunks'].apply(create_feature_vector)
test_df['features'] = test_df['chunks'].apply(create_feature_vector)

In [ ]:
len(test_df['features'][150])

400

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(df['features'], df['Author_name'], test_size=0.3, stratify=df['Author_name'], random_state=42)

X_train = np.array(df['features'].tolist())
X_test = np.array(test_df['features'].tolist())

y_train = df['Author_name']
y_test = test_df['Author_name']

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

le = LabelEncoder()
y_test_encoded = le.fit_transform(y_test)

# create and train the SVM model
svm = SVC(kernel='linear')
svm.fit(X_train, y_train_encoded)

# Make predictions on the test set and calculate evaluation metrics
y_pred = svm.predict(X_test)
accuracy = accuracy_score(y_test_encoded, y_pred)
f1 = f1_score(y_test_encoded, y_pred, average='weighted')
precision = precision_score(y_test_encoded, y_pred, average='weighted')
recall = recall_score(y_test_encoded, y_pred, average='weighted')

# Print evaluation metrics
print("SVM accuracy:", accuracy)
print("SVM F1-score:", f1)
print("SVM Precision:", precision)
print("SVM Recall:", recall)

SVM accuracy: 0.8847023360964582
SVM F1-score: 0.8825391585773119
SVM Precision: 0.8858277382624394
SVM Recall: 0.8847023360964582


In [ ]:
print(classification_report(y_test_encoded, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.85      0.86       567
           1       0.59      0.81      0.68       237
           2       0.95      0.87      0.91       736
           3       0.29      0.16      0.21        91
           4       0.97      1.00      0.98      1023

    accuracy                           0.88      2654
   macro avg       0.73      0.74      0.73      2654
weighted avg       0.89      0.88      0.88      2654



# num = 20 has the best scores

In [ ]:
num = 500

SVM accuracy: 0.8847023360964582
SVM F1-score: 0.8825391585773119
SVM Precision: 0.8858277382624394
SVM Recall: 0.8847023360964582

              precision    recall  f1-score   support

           0       0.88      0.85      0.86       567
           1       0.59      0.81      0.68       237
           2       0.95      0.87      0.91       736
           3       0.29      0.16      0.21        91
           4       0.97      1.00      0.98      1023

    accuracy                           0.88      2654
   macro avg       0.73      0.74      0.73      2654
weighted avg       0.89      0.88      0.88      2654


num = 250

SVM accuracy: 0.8700075357950263
SVM F1-score: 0.8693775564898865
SVM Precision: 0.8751643202875911
SVM Recall: 0.8700075357950263

num = 30

SVM accuracy: 0.8869630746043707
SVM F1-score: 0.8856245092582672
SVM Precision: 0.8875315458640809
SVM Recall: 0.8869630746043707


              precision    recall  f1-score   support

           0       0.89      0.87      0.88       567
           1       0.62      0.79      0.70       237
           2       0.94      0.86      0.90       736
           3       0.27      0.20      0.23        91
           4       0.97      1.00      0.98      1023

    accuracy                           0.89      2654
   macro avg       0.74      0.74      0.74      2654
weighted avg       0.89      0.89      0.89      2654



num = 15

SVM accuracy: 0.8944988696307461
SVM F1-score: 0.8913588454906688
SVM Precision: 0.8924540985045486
SVM Recall: 0.8944988696307461

              precision    recall  f1-score   support

           0       0.90      0.89      0.89       567
           1       0.62      0.80      0.70       237
           2       0.94      0.88      0.91       736
           3       0.32      0.18      0.23        91
           4       0.97      1.00      0.98      1023

    accuracy                           0.89      2654
   macro avg       0.75      0.75      0.74      2654
weighted avg       0.89      0.89      0.89      2654


num = 20

SVM accuracy: 0.8937452901281085
SVM F1-score: 0.891300001755393
SVM Precision: 0.8931268021007299
SVM Recall: 0.8937452901281085

              precision    recall  f1-score   support

           0       0.89      0.89      0.89       567
           1       0.62      0.81      0.70       237
           2       0.95      0.87      0.91       736
           3       0.31      0.19      0.23        91
           4       0.97      1.00      0.98      1023

    accuracy                           0.89      2654
   macro avg       0.75      0.75      0.74      2654
weighted avg       0.89      0.89      0.89      2654




num = 125

SVM accuracy: 0.8703843255463452
SVM F1-score: 0.8698806773574824
SVM Precision: 0.8752445856367435
SVM Recall: 0.8703843255463452

              precision    recall  f1-score   support

           0       0.89      0.85      0.87       567
           1       0.56      0.77      0.65       237
           2       0.94      0.83      0.88       736
           3       0.20      0.14      0.17        91
           4       0.95      1.00      0.97      1023

    accuracy                           0.87      2654
   macro avg       0.71      0.72      0.71      2654
weighted avg       0.88      0.87      0.87      2654


num = 60

SVM accuracy: 0.8820648078372268
SVM F1-score: 0.8792047159237686
SVM Precision: 0.8808336584289712
SVM Recall: 0.8820648078372268


             precision    recall  f1-score   support

           0       0.89      0.88      0.88       567
           1       0.60      0.79      0.68       237
           2       0.93      0.85      0.89       736
           3       0.23      0.13      0.17        91
           4       0.97      1.00      0.98      1023

    accuracy                           0.88      2654
   macro avg       0.72      0.73      0.72      2654
weighted avg       0.88      0.88      0.88      2654




num = 50

SVM accuracy: 0.888093443858327
SVM F1-score: 0.8845682514477369
SVM Precision: 0.8849243876334492
SVM Recall: 0.888093443858327

              precision    recall  f1-score   support

           0       0.89      0.89      0.89       567
           1       0.61      0.80      0.69       237
           2       0.93      0.87      0.90       736
           3       0.20      0.11      0.14        91
           4       0.97      1.00      0.98      1023

    accuracy                           0.89      2654
   macro avg       0.72      0.73      0.72      2654
weighted avg       0.88      0.89      0.88      2654


num = 100

SVM accuracy: 0.8854559155990958
SVM F1-score: 0.8838424335084509
SVM Precision: 0.8869829158977004
SVM Recall: 0.8854559155990958


              precision    recall  f1-score   support

           0       0.90      0.87      0.89       567
           1       0.60      0.81      0.69       237
           2       0.94      0.85      0.90       736
           3       0.27      0.18      0.21        91
           4       0.96      1.00      0.98      1023

    accuracy                           0.89      2654
   macro avg       0.73      0.74      0.73      2654
weighted avg       0.89      0.89      0.88      2654

num = 40

SVM accuracy: 0.8888470233609645
SVM F1-score: 0.8873166028424269
SVM Precision: 0.890844985715929
SVM Recall: 0.8888470233609645


             precision    recall  f1-score   support

           0       0.90      0.89      0.89       567
           1       0.60      0.81      0.69       237
           2       0.94      0.85      0.90       736
           3       0.26      0.16      0.20        91
           4       0.97      1.00      0.98      1023

    accuracy                           0.89      2654
   macro avg       0.74      0.74      0.73      2654
weighted avg       0.89      0.89      0.89      2654


num = 75

SVM accuracy: 0.8794272795779955
SVM F1-score: 0.8760996891274726
SVM Precision: 0.8777521798591438
SVM Recall: 0.8794272795779955

              precision    recall  f1-score   support

           0       0.88      0.85      0.87       567
           1       0.61      0.81      0.70       237
           2       0.93      0.85      0.89       736
           3       0.22      0.12      0.15        91
           4       0.96      1.00      0.98      1023

    accuracy                           0.88      2654
   macro avg       0.72      0.73      0.72      2654
weighted avg       0.88      0.88      0.88      2654

